# Algoritmi Predittivi

## Laboratorio Python

### Esperimento 1: Predizione della recidiva parte A: creazione di un dataset di addestramento e test simulato {#sec-esperimento-1-predizione-della-recidiva-parte-a}

Vediamo un esempio in Python che implementa il processo predittivo per il caso giuridico di previsione della probabilità di recidiva utilizzando una **Random Forest Regressor**. Questo esempio utilizza diviso in 3 parti si basa su dati simulati per illustrare il flusso completo, dalla generazione dei dati fino alla valutazione del modello.

::: {.callout-caution}
Se non è stato già fatto, installare le librerie necessarie con il seguente comando da eseguire nella shell:
```
pip install numpy pandas sklearn matplotlib
```
:::

In questa primo esperimento l'obiettivo principale è generare dati dall'aspetto realistico su persone che sono in libertà vigilata (rilascio supervisionato dopo un reato) e calcolare il loro rischio di recidiva. Questi dati simulati verranno successivamente utilizzati per addestrare e testare un modello predittivo, che è una pratica comune nel machine learning quando si vuole sperimentare prima di utilizzare dati reali e sensibili.
Il codice non riceve input esterni dagli utenti. Invece, genera i propri dati utilizzando la generazione di numeri casuali. Tuttavia, utilizza diversi parametri predefiniti per controllare la simulazione:

- Numero di campioni (1000 persone)
- Fasce d'età (da 18 a 70 anni)
- Durata della libertà vigilata (da 1 a 60 mesi)
- Tipi di reati precedenti (furto, reato violento, omicidio)
- Stato lavorativo (lavoro stabile o no)
- Supporto familiare (presente o assente)

Il codice segue una sequenza chiara: genera caratteristiche individuali casualmente, poi le combina usando una formula ponderata per calcolare il rischio. Un passaggio cruciale è il "clipping" del punteggio di rischio finale per assicurarsi che rimanga tra 0 e 1, poiché le probabilità non possono essere negative o maggiori del 100%.

Il seed casuale (impostato a 42) garantisce che ogni volta che esegui questo codice, ottieni esattamente gli stessi dati "casuali", il che è importante per esperimenti scientifici riproducibili. Infine, tutti i dati generati vengono organizzati in un DataFrame pandas (pensalo come un foglio di calcolo digitale) e visualizza le prime righe così puoi vedere com'è fatto il dataset simulato.

Questo approccio permette a ricercatori e sviluppatori di testare i loro algoritmi di predizione su dati realistici senza problemi di privacy, prima di applicarli a veri fascicoli giudiziari.


In [ ]:
import numpy as np
import pandas as pd

# Simulazione dei dati con relazioni realistiche
np.random.seed(42)
num_samples = 1000

# Generazione delle variabili
eta = np.random.randint(18, 70, size=num_samples)
durata_liberta_vigilata = np.random.randint(1, 60, size=num_samples)  # Durata in mesi
reato_precedente = np.random.choice([0, 1, 2], size=num_samples, p=[0.5, 0.3, 0.2])  # Furto, violenza privata, omicidio
lavoro_stabile = np.random.choice([0, 1], size=num_samples, p=[0.6, 0.4])
supporto_familiare = np.random.choice([0, 1], size=num_samples, p=[0.4, 0.6])

# Calcolo del rischio di recidiva basato su regole
rischio_recidiva = (
    0.6 * (1 - lavoro_stabile) +
    0.5 * (1 - supporto_familiare) +
    0.4 * (reato_precedente / 2) +
    0.3 * (1 / durata_liberta_vigilata) +
    0.2 * (1 / (eta - 17))
)
rischio_recidiva = np.clip(rischio_recidiva, 0, 1)  # Limitare il rischio tra 0 e 1

# Creazione del DataFrame
data = {
    'eta': eta,
    'durata_liberta_vigilata': durata_liberta_vigilata,
    'reato_precedente': reato_precedente,
    'lavoro_stabile': lavoro_stabile,
    'supporto_familiare': supporto_familiare,
    'rischio_recidiva': rischio_recidiva
}

df = pd.DataFrame(data)

# Visualizzazione iniziale dei dati
print("Esempio di dati simulati:")
print(df.head())

Si noti che i dati generati sono memorizzati in una struttura dati chiamata Dataframe.
Si tratta di una particolare struttura dati fornita dalla libreria pandas di Python che rappresenta essenzialmente una tabella, molto simile a un foglio di calcolo di Excel o a una tabella di un database. 
Possiamo immaginarla come una griglia con righe e colonne: ogni riga rappresenta un record (in questo caso, una persona) e ogni colonna rappresenta una caratteristica o variabile (come età, tipo di reato, ecc.).
Questa struttura dati è diventata molto popolare nel mondo del machine learning perché è molto flessibile e può gestire una grande quantità di dati in modo efficiente.
Con questa struttura dati, è possibile eseguire operazioni come la selezione di righe o colonne specifiche, la manipolazione dei dati, la creazione di nuovi dati basati su quelli esistenti e molto altro.
Inoltre, le librerie di machine learning come scikit-learn richiedono spesso che i dati siano organizzati in questo formato per poter essere utilizzati.

### Esperimento 2: Predizione della recidiva parte B: Addestramento e test del modello

In questo esperimento, il modello Random Forest Regressor viene addestrato utilizzando i dati simulati e quindi viene utilizzato per fare previsioni sui dati di test.
Per prima cosa, i dati sono divisi in input (X) e output (y), dove X contiene tutte le caratteristiche (ad esempio, età, tipo di reato precedente, ecc.) e y contiene il rischio di recidiva.
Successivamente, i dati sono divisi in set di addestramento e test utilizzando la funzione train_test_split di scikit-learn.
Il modello Random Forest Regressor viene addestrato utilizzando i dati di addestramento e quindi viene utilizzato per fare previsioni sui dati di test.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Divisione dei dati in input (X) e output (y)
X = df.drop('rischio_recidiva', axis=1)
y = df['rischio_recidiva']

# Divisione in training e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creazione e addestramento del modello Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Predizione sui dati di test
y_pred = rf_model.predict(X_test)

# Valutazione del modello
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nRisultati del modello:")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")

### Esperimento 3: Predizione della recidiva parte C: Ottimizzazione degli iperparametri del modello e visualizzazione dei risultati

In questa parte dell'esperimento, viene visualizzato un grafico per confrontare i valori reali di rischio di recidiva con i valori predetti dal modello.
Inoltre, viene utilizzato GridSearchCV per ottimizzare gli iperparametri del modello Random Forest Regressor.
Questo processo di ottimizzazione è importante perché aiuta a trovare la combinazione migliore di parametri che migliorano le prestazioni del modello.
Infine, i risultati dell'ottimizzazione vengono visualizzati in un grafico.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

# Visualizzazione dei risultati
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([0, 1], [0, 1], '--', color='red', label='Predizione Perfetta')
plt.xlabel("Valori Reali di Rischio di Recidiva")
plt.ylabel("Valori Predetti di Rischio di Recidiva")
plt.title("Confronto tra Valori Reali e Predetti")
plt.legend()
plt.grid()
plt.show()

# Ottimizzazione degli iperparametri con GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("\nMigliori parametri trovati con GridSearchCV:")
print(grid_search.best_params_)

# Addestramento del modello ottimizzato
best_rf_model = grid_search.best_estimator_

# Predizione e valutazione con il modello ottimizzato
y_pred_optimized = best_rf_model.predict(X_test)
optimized_mse = mean_squared_error(y_test, y_pred_optimized)
optimized_r2 = r2_score(y_test, y_pred_optimized)

# Visualizzazione dei risultati
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_optimized, alpha=0.7)
plt.plot([0, 1], [0, 1], '--', color='red', label='Predizione Perfetta')
plt.xlabel("Valori Reali di Rischio di Recidiva")
plt.ylabel("Valori Predetti di Rischio di Recidiva")
plt.title("Confronto tra Valori Reali e Predetti  con parametri ottimizzati")
plt.legend()
plt.grid()
plt.show()

print("\nRisultati del modello ottimizzato:")
print(f"Mean Squared Error (MSE): {optimized_mse:.4f}")
print(f"R^2 Score: {optimized_r2:.4f}")